# Whereabouts Plans Template
This notebook will demonstrate how to create a whereabouts plan automatically.

<div style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg/640px-Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg" /></div>

## Introduction
The purpose of this notebook is to create a Street and Bridge Work Order plans based on segment IDs and additional comments on long line. Markings feature layers are published in the City of Austin ArcGIS Portal page available for public view as well. 

The data should already be available in the folder path as an excel spreadsheet. If the spreadsheet exists, a map document will be configured for spreadsheet use.

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [pathlib](https://docs.python.org/3/library/pathlib.html) to find path to excel document if it exists
- [archook](https://github.com/JamesRamm/archook) to search for arcgis and makes arcpy available to python
- [arcpy](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm) to create whereabouts markings plans using ESRI ArcMap Desktop software
- [openpyxl](https://openpyxl.readthedocs.io/en/stable/) to open excel files of markings and SBO data

In [9]:
import pandas as pd
from pathlib import Path
import datetime
import math
from functools import reduce

import archook 
archook.get_arcpy()
import arcpy

from PyPDF2 import PdfFileMerger
import img2pdf
import openpyxl
from PIL import Image 
import os 

## Constants

The date by month and day constant will determine the file pdf name to use as a dataframe. Folder path will determine where the plans will be created depending on the year. This is set to the top for the purpose of changing these constants as needed.

<i>The table below explains the purpose of each constant.</i>

| Constant | Description   |
|:--------:|----|
| <b>MONTH, DAY, YEAR</b> |Date used to find PDF in month-day format and file path based on year|
|<b>FOLDER</b>      |File directory used to import SBO whereabouts reports from email|
|<b>FILE_NAME</b>   |File directory name used to extact SBO whereabouts reports from file|
|<b>MXD</b>   |Map document used to create whereabouts plans template|

In [23]:
%store -r MONTH
%store -r DAY
%store -r YEAR
%store -r FOLDER
%store -r FILE_NAME
MXD = r"G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\template\Whereabouts_Cover_Template.mxd"
PLAN_PATH = r"G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\{}".format(YEAR)

## Methods
These functions will be used to extract and transform the data into a feasible format.

<i>The table below explains the purpose of each:</i>

| Method | Description   |
|:--------:|----|
|<b>df_text</b> |Changes element text to the pandas dataframe|
|<b>cover_aerial</b> |Zooms to selected features and extent for the dataframe|

In [17]:
# Formats element text to match pandas dataframe
def df_text(r):
    date = datetime.datetime.now()
    for e in arcpy.mapping.ListLayoutElements(mxd,'TEXT_ELEMENT'):
            if e.name == 'SPECIFICATIONS':
                e.text = str(r["Specifications"])
            elif e.name == 'CREATED DATE':
                e.text =  "{}/{}/{}".format(date.month,date.day,date.year)
            elif e.name == 'REQUESTOR ID':
                e.text = str(r["Location ID"])
            elif e.name == 'LOCATION':
                e.text = "{} from {} to {}".format(r["Street"],r["From"],r["To"])
            elif e.name == 'author':
                e.text = 'Susanne Gov'
            elif e.name == 'WORK GROUPS':
                e.text = str(r['WORK GROUPS'])

# Refreshes dataframe map aerial                
def cover_aerial(mapdoc):
    dataframe = arcpy.mapping.ListDataFrames(mapdoc,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    arcpy.RefreshActiveView()

## Import Datasets

In [25]:
if Path(FILE_NAME + '.xlsx').exists():
    df = pd.read_excel(FILE_NAME + '.xlsx','markings list', index_col=0).reset_index()
    display(df)

,Location ID,Street,From,To,Segment IDs,Comments,Crosswalk,Stopline,Only word,Left arrow,Right arrow,Bike arrow,Bicyclist symbol,Diagonal crosshatch,WORK GROUPS,Specifications
0,62870,DAVIS LN,4304,Mo-Pac Svc Rd Nb,"2026275,2038344,2038345,2043055,3259330,325933...","lane lines, bike lanes, turn bay",1.0,2.0,1.0,NaN,1.0,21.0,8.0,10.0,"['SHORT LINE', 'LONGLINE', 'SPECIALTY MARKINGS']","Install lane lines, bike lanes, turn bay1 cros..."
1,62872,DEER LN,4000,4313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Install N/A
2,Lakeshore,Royal Crest,0.5 LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Install N/A
3,WALL ST,Cross Park Dr,Ferguson Ln,3.89,2040261,"turn bay, lane lines, double yellow dashed sol...",2.0,2.0,1.0,4.0,2.0,NaN,NaN,13.0,"['SHORT LINE', 'LONGLINE', 'SPECIALTY MARKINGS']","Install turn bay, lane lines, double yellow da..."


## Setting up Cover Document
A database connection to GISDM is needed to access the markings asset layers and the markings assets layers. We can open up a map document of the cover work orders and another map document of the pages work orders.

In [26]:
mxd = arcpy.mapping.MapDocument(MXD)
sde_path = r"Database Connections\GISDM.sde"

# Create Database connection 
arcpy.env.workspace = sde_path
if arcpy.Exists(sde_path) == False:
    arcpy.CreateDatabaseConnection_management("Database Connections","GISDM.sde", "ORACLE", 
                                          "sde:oracle11g:gisdm", "DATABASE_AUTH")

In [27]:
arcpy.env.workspace = sde_path
if arcpy.Exists(sde_path) == False:
    arcpy.CreateDatabaseConnection_management("Database Connections","GISDM.sde", "ORACLE", 
                                          "sde:oracle11g:gisdm", "DATABASE_AUTH")
layer = arcpy.mapping.ListLayers(mxd,"TRANSPORTATION.street_segment")[0]
for index,row in df.iterrows():
    segments = str(row["Segment IDs"]).split(',')
    sql = 'SEGMENT_ID IN({})'.format(str(segments)[1:-1])
    print sql
    arcpy.SelectLayerByAttribute_management(layer,"NEW_SELECTION",sql) 
    for l in arcpy.mapping.ListLayers(mxd,'TRANSPORTATION.markings*'):
        #arcpy.SelectLayerByAttribute_management(l,"NEW_SELECTION",sql) 
        pass
    df_text(row)
    cover_aerial(mxd)
    arcpy.mapping.ExportToPDF(mxd, PLAN_PATH + "\\ATD_Whereabouts_Cover_{}.pdf".format(row["Location ID"]))
    arcpy.SelectLayerByAttribute_management(layer,"CLEAR_SELECTION")
    print "PDF file created Cover {}.pdf".format(row["Location ID"])

SEGMENT_ID IN('2026275', '2038344', '2038345', '2043055', '3259330', '3259331', '3259334', '3259335')
PDF file created Cover 62870.pdf
SEGMENT_ID IN('nan')


ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


# Setting up Pages

In [91]:
layer = arcpy.mapping.ListLayers(mxd,'TRANSPORTATION.markings_short_line')[0]
arcpy.SelectLayerByAttribute_management(arcpy.mapping.ListLayers(mxd,"TRANSPORTATION.street_segment")[0],"CLEAR_SELECTION")
df1['COUNT'] = 1
pages_sl = df1.groupby(['LOCATION ID',"SEGMENT ID","SHORTLINE TYPE"]).count()[['COUNT']].reset_index()
pages_sl['id'] = pages_sl['LOCATION ID'].astype(int).astype(str) + '_' + pages_sl['SEGMENT ID'].astype(int).astype(str)

pages_sp = df1.groupby(['LOCATION ID',"SEGMENT ID",'SPECIALTY TYPE','SPECIALTY SUBTYPE']).count()[['COUNT']].reset_index()
pages_sp['SPECIALTY'] = pages_sp['SPECIALTY TYPE'].astype(int).astype(str) + ',' + pages_sp['SPECIALTY SUBTYPE'].astype(int).astype(str)
pages_sp = pages_sp.drop(columns=['SPECIALTY TYPE','SPECIALTY SUBTYPE'])
pages_sp['id'] = pages_sp['LOCATION ID'].astype(int).astype(str) + '_' + pages_sp['SEGMENT ID'].astype(int).astype(str)

pages = pd.merge(pages_sp,pages_sl, on='id',how='outer')

for index,row in pages.iterrows():
    segment = int(row["SEGMENT ID_x"])
    q = 'SEGMENT_ID {}'.format(segment)
    print q
    arcpy.SelectLayerByAttribute_management(layer, "NEW_SELECTION", q)
    for e in arcpy.mapping.ListLayoutElements(mxd,'TEXT_ELEMENT'):
            if e.name == 'SPECIFICATIONS':
                e.text = "Install {} {}s".format(row['COUNTS'],row['SHORTLINE TYPE'].lower().replace("_"," "))
            elif e.name == 'REQUESTOR ID':
                e.text = str(row["LOCATION ID"])
            elif e.name == 'row1':
                e.text = 'SEGMENT ID'
            elif e.name == 'LOCATION':
                e.text = str(row["SEGMENT_ID"])
    dataframe = arcpy.mapping.ListDataFrames(mxd,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    dataframe.scale = 1000
    arcpy.RefreshActiveView()
    arcpy.mapping.ExportToPDF(mxd, PLAN_PATH + "\\ATD_Whereabouts_Page_{}.pdf".format(row["id"]))
    print "PDF file created Page_{}.pdf".format(row["id"])

SEGMENT_ID IN(2010547)


ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


In [93]:
for l in arcpy.mapping.ListLayers(mxd,'TRANSPORTATION.markings*'):
    #if arcpy.Exists(l) == False:
    #    print 'why'
    print l

TRANSPORTATION.markings_specialty_point
TRANSPORTATION.markings_specialty_line
TRANSPORTATION.markings_short_line


In [ ]:
a = PLAN_PATH + "\\ATD_Whereabouts_"

for i,r in df.iterrows():
    merger = PdfFileMerger()
    pdf = "{}{}_{}.pdf".format(a,'Cover',r['Location ID'])
    merger.append(pdf)
    for index,row in pages.iterrows():
        if r['Location ID'] == row['LOCATION ID']:
            png = "{}{}_{}_{}.png".format(a,'Page',row['LOCATION ID'],row['SEGMENT_ID'])
            rgba = Image.open(png)
            rgb = Image.new('RGB', rgba.size, (255, 255, 255))  # white background
            rgb.paste(rgba, mask=rgba.split()[3])               # paste using alpha channel as mask
            rgb.save(PDF_FILE, 'PDF', resoultion=100.0)
            pdf = "{}{}_{}_{}.pdf".format(a,'Page',row['LOCATION ID'],row['SEGMENT_ID'])
            merger.append(pdf)
    merger.write(a + "{}_Final.pdf".format(r['Location ID']))
    merger.close()

In [8]:
print df.Specifications[6]

Install turn bays, lane line 2 crosswalk, 2 stopline, 1 only word, 3 left  arrow, 2 right arrow, 3 bike arrow, 1 bicyclist symbol
